<a href="https://colab.research.google.com/github/souravs17031999/private-ai/blob/master/Final_encrypted_dl_project_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project - XVII
## Objective : to implement deep learning techniques using Pysyft on the testing / inference of model so that our data as well as model parameters is encrypted while our model can be used by anyone for their own inference.

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
!pip install syft

In [0]:
import syft as sy  
hook = sy.TorchHook(torch) 
client = sy.VirtualWorker(hook, id="client") 
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
crypto_provider = sy.VirtualWorker(hook, id="crypto_provider") 

W0804 06:54:01.520527 140384489969536 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0804 06:54:01.545021 140384489969536 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 200
        self.epochs = 10
        self.lr = 0.001 # learning rate
        self.log_interval = 100

args = Arguments()

In [0]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True)

0it [00:00, ?it/s]

9920512it [00:01, 8298594.64it/s]                            


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 136345.60it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 2202234.24it/s]                            
0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 52082.94it/s]            


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True)

# Convert to integers and privately share the dataset
private_test_loader = []
for data, target in test_loader:
    private_test_loader.append((
        data.fix_prec().share(alice, bob, crypto_provider=crypto_provider),
        target.fix_prec().share(alice, bob, crypto_provider=crypto_provider)
    ))

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [0]:
def train(args, model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        output = F.log_softmax(output, dim=1)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size,
                100. * batch_idx / len(train_loader)))

In [0]:
model = Net()
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

for epoch in range(1, args.epochs + 1):
    train(args, model, train_loader, optimizer, epoch)

Train Epoch: 1 [0/60032 (0%)]
Train Epoch: 1 [6400/60032 (11%)]
Train Epoch: 1 [12800/60032 (21%)]
Train Epoch: 1 [19200/60032 (32%)]
Train Epoch: 1 [25600/60032 (43%)]
Train Epoch: 1 [32000/60032 (53%)]
Train Epoch: 1 [38400/60032 (64%)]
Train Epoch: 1 [44800/60032 (75%)]
Train Epoch: 1 [51200/60032 (85%)]
Train Epoch: 1 [57600/60032 (96%)]
Train Epoch: 2 [0/60032 (0%)]
Train Epoch: 2 [6400/60032 (11%)]
Train Epoch: 2 [12800/60032 (21%)]
Train Epoch: 2 [19200/60032 (32%)]
Train Epoch: 2 [25600/60032 (43%)]
Train Epoch: 2 [32000/60032 (53%)]
Train Epoch: 2 [38400/60032 (64%)]
Train Epoch: 2 [44800/60032 (75%)]
Train Epoch: 2 [51200/60032 (85%)]
Train Epoch: 2 [57600/60032 (96%)]
Train Epoch: 3 [0/60032 (0%)]
Train Epoch: 3 [6400/60032 (11%)]
Train Epoch: 3 [12800/60032 (21%)]
Train Epoch: 3 [19200/60032 (32%)]
Train Epoch: 3 [25600/60032 (43%)]
Train Epoch: 3 [32000/60032 (53%)]
Train Epoch: 3 [38400/60032 (64%)]
Train Epoch: 3 [44800/60032 (75%)]
Train Epoch: 3 [51200/60032 (85%)]
Tra

In [0]:
model.fix_precision().share(alice, bob, crypto_provider=crypto_provider)


Net(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [0]:
def test(args, model, test_loader):
    model.eval()
    n_correct_priv = 0
    n_total = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            pred = output.argmax(dim=1) 
            n_correct_priv += pred.eq(target.view_as(pred)).sum()
            n_total += args.test_batch_size
            
            n_correct = n_correct_priv.copy().get().float_precision().long().item()
    
            print('Test set: Accuracy: {}/{} ({:.0f}%)'.format(
                n_correct, n_total,
                100. * n_correct / n_total))

In [0]:
test(args, model, private_test_loader)


Test set: Accuracy: 195/200 (98%)
Test set: Accuracy: 392/400 (98%)
Test set: Accuracy: 590/600 (98%)
Test set: Accuracy: 784/800 (98%)
Test set: Accuracy: 977/1000 (98%)
Test set: Accuracy: 1172/1200 (98%)
Test set: Accuracy: 1369/1400 (98%)
Test set: Accuracy: 1561/1600 (98%)
Test set: Accuracy: 1758/1800 (98%)
Test set: Accuracy: 1957/2000 (98%)
Test set: Accuracy: 2155/2200 (98%)
Test set: Accuracy: 2343/2400 (98%)
Test set: Accuracy: 2539/2600 (98%)
Test set: Accuracy: 2735/2800 (98%)
Test set: Accuracy: 2931/3000 (98%)
Test set: Accuracy: 3127/3200 (98%)
Test set: Accuracy: 3324/3400 (98%)
Test set: Accuracy: 3519/3600 (98%)
Test set: Accuracy: 3714/3800 (98%)
Test set: Accuracy: 3912/4000 (98%)
Test set: Accuracy: 4107/4200 (98%)
Test set: Accuracy: 4304/4400 (98%)
Test set: Accuracy: 4501/4600 (98%)
Test set: Accuracy: 4693/4800 (98%)
Test set: Accuracy: 4887/5000 (98%)
Test set: Accuracy: 5083/5200 (98%)
Test set: Accuracy: 5281/5400 (98%)
Test set: Accuracy: 5476/5600 (98%)
T